In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random

In [4]:
cm = 200
g_L = 10
E_L = -70
E_ex = 0
E_inh = -70
tau_ex = 6
delta_g_ex = 12.5 #1.25 #0.01
g_ex = 0
delta_g_inh = 30 #3 #0.01
g_inh = 0
tau_inh = 3
V_T = -50
V_thr = 0
V_reset = -58
mn_b = 60
mn_a = 0
mn_tau_w = 30

time = 5000
delta_t = 0.1
n_ains = 1

In [ ]:
ain_g_ex = np.zeros((n_ains,1))
ain_g_inh = np.zeros((n_ains,1))
ain_w = np.zeros((n_ains,1))

ain_voltage_vec = np.zeros((n_ains,1))
ain_voltage_vec[:,:] += E_L
ain_voltage_record = np.zeros((1,n_timesteps))
ain_spike_times = [[] for m in range(n_ains)]

Is = []

In [ ]:
for I in Is:
    for idx,t in enumerate(np.arange(0,time,delta_t)):
        for ain in range(n_ains):
            if ain_voltage_vec[ain,-1] >= V_thr:
                ain_voltage_vec[ain,-1] = V_reset
        ain_voltage_vec[:,-1] += delta_t*((-g_L*(ain_voltage_vec[:,-1] - E_L) + g_L * slope_f * np.exp((ain_voltage_vec[:,-1] - V_T)/slope_f) \
                                            - ain_g_ex[:,-1] * (ain_voltage_vec[:,-1] - E_ex) \
                                            - ain_g_inh[:,-1] * (ain_voltage_vec[:,-1] - E_inh) + I_L - ain_w[:,-1]) / cm)
        ain_w += delta_t*((mn_a * (ain_voltage_vec - E_L) - ain_w) / mn_tau_w)
        ain_g_ex += delta_t*(-ain_g_ex/tau_ex)
        ain_g_inh += delta_t*(-ain_g_inh/tau_inh)
        for ain in range(n_ains):
            if ain_voltage_vec[ain,-1] >= V_thr:
                ain_voltage_vec[ain,-1] = V_thr
                ain_spike_times[ain].append(t)
                ain_w[ain,-1] += mn_b
        ain_voltage_record[:,idx] += ain_voltage_vec[[0,int(n_ains/2)],:].squeeze()
    
x = np.arange(0,time,delta_t) 

for i in range(2):
    plt.plot(x,mn_voltage_record[i,:],label='MN')
    plt.plot(x,ain_voltage_record[i,:],label='aIN')
    plt.plot(x,cin_voltage_record[i,:],label='cIN')
    plt.plot(x,din_voltage_record[i,:],label='dIN')
plt.title('AdEx CPG',fontsize='16')
plt.xlabel('Time (ms)',fontsize='14')
plt.ylabel('Voltage (mV)',fontsize='14')
ain_w = np.zeros((n_ains,1))
plt.legend()
# plt.savefig('adexcpg')